In [1]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
pip install tensorflow==2.4

ERROR: Could not find a version that satisfies the requirement tensorflow==2.4 (from versions: none)
ERROR: No matching distribution found for tensorflow==2.4
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.preprocessing import StandardScaler

In [2]:
from tensorflow import keras

In [6]:
from tensorflow.keras.models import Sequential

In [11]:
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense

In [18]:
from tensorflow.keras import layers

In [21]:
import numpy as np
import pandas as pd
import time

In [7]:
import matplotlib.pyplot as plt
import matplotlib as cm

In [3]:
import statsmodels.api as sm 

In [8]:
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import BatchNormalization



In [9]:
np.__version__

'1.22.3'

In [10]:
X_train = np.load('X_train_surge.npz')
Y_train = pd.read_csv('Y_train_surge.csv')

In [11]:
id_seq=X_train['id_sequence']
t_slp=X_train['t_slp']
slp=X_train['slp']
t_surge1_input=X_train['t_surge1_input']
surge1_input=X_train['surge1_input']
t_surge2_input=X_train['t_surge2_input']
surge2_input=X_train['surge2_input']
t_surge1_output=X_train['t_surge1_output']
t_surge2_output=X_train['t_surge2_output']

array([-39616., -39616., -35968., -36032., -35968., -36032., -32384.,
       -32384., -28800., -28800.], dtype=float32)

array([-6.309792e+08, -6.309360e+08, -6.308928e+08, -6.308496e+08,
       -6.308064e+08, -6.307632e+08, -6.307200e+08, -6.306768e+08,
       -6.306336e+08, -6.305904e+08], dtype=float32)

array([-6.310188e+08, -6.309756e+08, -6.309288e+08, -6.308856e+08,
       -6.308424e+08, -6.307992e+08, -6.307524e+08, -6.307092e+08,
       -6.306624e+08, -6.306192e+08], dtype=float32)

array([  43200.,   32384.,   21568.,   10816.,       0.,  -10816.,
        -21632.,  -32448.,  -43200.,  -54016.,  -64832.,  -75584.,
        -86400.,  -97216., -108032., -118848., -129600., -140416.,
       -151232., -161984., -172800., -183616., -194432., -205248.,
       -216000., -226816., -237632., -248384., -259200., -270016.,
       -280832., -291648., -302400., -313216., -324032., -334784.,
       -345600., -356416., -367232., -378048.], dtype=float32)

time.struct_time(tm_year=1950, tm_mon=1, tm_mday=2, tm_hour=12, tm_min=59, tm_sec=44, tm_wday=0, tm_yday=2, tm_isdst=0)

array([ -9.9911108 ,   1.99111116,  14.0088892 ,  25.99111176,
        39.00444412,  51.00444412,  64.        ,  76.        ,
        88.        , 100.        ])

array([ -8.99555588,   3.00444436,  16.        ,  28.        ,
        40.        ,  52.99555588,  64.99555206,  76.99555206,
        89.99111176, 102.00888824])

array([  0.        ,   3.00444436,   6.00888872,   8.99555588,
        12.        ,  15.00444412,  17.99111176,  20.99555588,
        24.        ,  27.00444412,  30.00888824,  32.99555588,
        36.        ,  39.00444412,  41.99111176,  44.99555588,
        48.        ,  51.00444412,  54.00888824,  56.99555588,
        60.        ,  63.00444412,  65.99111176,  68.99555206,
        72.        ,  75.00444794,  78.00888824,  80.99555206,
        84.        ,  87.00444794,  89.99111176,  92.99555206,
        96.        ,  99.00444794, 102.00888824, 104.99555206,
       108.        , 111.00444794, 113.99111176, 116.99555206])

One thing that can be done is to use a LSTM model with multivariate entrance and use every 10 input sea surge to predict the next 10, together with the averages of the slps. 

(5599, 40, 41, 41)

(5599, 10)

In [15]:
surge_train=np.array(Y_train)[:,1:] #surge_train[0:9] -> surge1
                                    #surge_train[10:19] -> surge2

In [15]:
surge1_input.shape

(5599, 10)

In [14]:
surge_train.shape

(5599, 20)

array([[1.05653211, 0.23217276],
       [0.23217276, 1.00849238]])

In [12]:
#data to prediction
nbatches=surge1_input.shape
data_input=[]
for i in range(len(surge1_input)):
    data_input.append(slp[i,-1:-9:-4])
data_input=np.array(data_input)
print(data_input.shape)

(5599, 2, 41, 41)


In [13]:
data_input2=np.zeros((data_input.shape[0],data_input.shape[1],data_input.shape[2]*data_input.shape[3]))
for i in range(len(data_input)):
    for j in range(data_input.shape[1]):
        data_input2[i,j]=np.ndarray.flatten(data_input[i,j])



In [22]:
data_input2.shape

(5599, 2, 1681)

In [16]:
#output data
data_output=surge_train


In [26]:
print(data_output.shape)

(5599, 20)


In [17]:
#scale the dataset
scaler=StandardScaler()
orig_shape=data_input2.shape
data_input2=data_input2.reshape((len(data_input2)*data_input2.shape[1],data_input2.shape[2]))
scaler=scaler.fit(data_input2)
data_input_training=scaler.transform(data_input2)
data_input_training=data_input_training.reshape(orig_shape)
data_input2=data_input2.reshape(orig_shape)

In [19]:
#Now I build the model 
n_features = data_input_training.shape[2]
n_steps_in = data_input_training.shape[1]
# define model
model = Sequential()
model.add(layers.LSTM(units=128, activation='relu',input_shape=(n_steps_in,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
#model.add(Dropout(0.2))
model.add(Dense(data_output.shape[1])
model.compile(optimizer='adam', loss='mse')
model.summary()

NotImplementedError: Cannot convert a symbolic Tensor (lstm_4/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [27]:
# fit model
model.fit(data_input, data_output, epochs=100, verbose=1)

NameError: name 'data_input' is not defined

In [ ]:
# Check the forecasting
x_input = data_input_training[0:20]
y_output = model.predict(x_input, verbose=1)